In [ ]:
import numpy as np
import pandas as pd
import walnut

# Example 5

### More complex convolutional neural network

The goal of this model is to classify images of clothing items.

### Step 1: Prepare data
You will need to download the dataset from https://www.kaggle.com/datasets/zalando-research/fashionmnist?resource=download and place it into the *data* directory. Only using the official training data for training, validation and testing, since it is just to showcase the framework.

In [ ]:
data = pd.read_csv('data/fashion_mnist/fashion-mnist_train.csv')
data.head()

In [ ]:
data_enc = walnut.preprocessing.encoding.pd_one_hot_encode(data, columns=['label'])
data_enc.head()

In [ ]:
tensor = walnut.pd_to_tensor(data_enc)
train, val, test = walnut.preprocessing.split_train_val_test(tensor, ratio_val=0.005, ratio_test=0.005)

In [ ]:
x_train, y_train = walnut.preprocessing.split_features_labels(train, 784)
x_val, y_val = walnut.preprocessing.split_features_labels(val, 784)
x_test, y_test = walnut.preprocessing.split_features_labels(test, 784)

In [ ]:
x_train = x_train.reshape((x_train.shape[0], 1 , 28, -1))
x_val = x_val.reshape((x_val.shape[0], 1, 28, -1))
x_test = x_test.reshape((x_test.shape[0], 1, 28, -1))

print (f'{x_train.shape=}')
print (f'{y_train.shape=}')

print (f'{x_val.shape=}')
print (f'{y_val.shape=}')

print (f'{x_test.shape=}')
print (f'{y_test.shape=}')

Normalization

In [ ]:
x_train = x_train / 255
x_val = x_val / 255
x_test = x_test / 255

### Step 2: Build the neural network structure

In [ ]:
import walnut.nn as nn
from walnut.nn.layers import *

model = nn.Sequential([
    Convolution2d(1, 16, kernel_size=(3, 3), pad="same", use_bias=False), Batchnorm(16), Relu(),
    Convolution2d(16, 16, kernel_size=(3, 3), pad="same", use_bias=False), Batchnorm(16), Relu(),
    MaxPooling2d(kernel_size=(2, 2)),
    Dropout(0.1),
    Convolution2d(16, 32, kernel_size=(3, 3), pad="same", use_bias=False), Batchnorm(32), Relu(),
    Convolution2d(32, 32, kernel_size=(3, 3), pad="same", use_bias=False), Batchnorm(32), Relu(),
    MaxPooling2d(kernel_size=(2, 2)),
    Dropout(0.1),
    Reshape(),
    Linear(7*7*32, 200, use_bias=False), Batchnorm(200), Relu(),
    Linear(200, 10), Softmax()
])

The network is compiled to internally connect it's layers and initialize the model.

In [ ]:
model.compile(
    optimizer=nn.optimizers.Adam(l_r=1e-3),
    loss_fn=nn.losses.Crossentropy(),
    metric=nn.metrics.Accuracy()
)

In [ ]:
model

### Step 3: Train the model

In [ ]:
epochs = 100
batch_size = 32

train_loss_hist, val_loss_hist = model.train(x_train, y_train, epochs=epochs, batch_size=batch_size, val_data=(x_val, y_val))

In [ ]:
n = epochs // 10  # average over the last n values to reduce noise caused by a small batch size

traces = {
    "train_loss" : [np.average(train_loss_hist[i-min(n-1, i):i+1]) for i in range(len(train_loss_hist))],
    "val_loss" : val_loss_hist
}

nn.analysis.plot_curve(traces=traces, figsize=(15, 3), title="loss history", x_label="epoch", y_label="loss")

### Step 4: Evaluate the model

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'loss {loss:.4f}')
print(f'accuracy {100*accuracy:.2f}')

In [ ]:
predictions = model(x_test)
nn.analysis.plot_confusion_matrix(predictions, y_test, figsize=(5, 5), cmap='Blues')